# General Imports

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

import pandas as pd
from google.colab import drive
from pathlib import Path
from warnings import simplefilter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor





drive.mount('/content/drive/')


In [ ]:
simplefilter("ignore")

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 4))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)

plot_params2 = dict(
    color="red",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)

%config InlineBackend.figure_format = 'retina'


def plot_multistep(y, every=1, ax=None, palette_kwargs=None):
    palette_kwargs_ = dict(palette='husl', n_colors=16, desat=None)
    if palette_kwargs is not None:
        palette_kwargs_.update(palette_kwargs)
    palette = sns.color_palette(**palette_kwargs_)
    if ax is None:
        fig, ax = plt.subplots()
    ax.set_prop_cycle(plt.cycler('color', palette))
    for date, preds in y[::every].iterrows():
        preds.index = pd.period_range(start=date, periods=len(preds), freq='H')
        preds.plot(ax=ax)
    return ax

# Dataset Pipelines

### Water Levels

In [ ]:
def prepare_df(dataf):
  dataf = (dataf
          .rename(columns={"관측 일시":"date", "01시":"H01", "02시":"H02", "03시":"H03", "04시":"H04", "05시":"H05", "06시":"H06", "07시":"H07", "08시":"H08", "09시":"H09", "10시":"H10","11시":"H11", "12시":"H12", "13시":"H13", "14시":"H14", "15시":"H15", "16시":"H16", "17시":"H17", "18시":"H18", "19시":"H19", "20시":"H20", "21시":"H21", "22시":"H22", "23시":"H23", "24시":"H24"})
          .drop([dataf.shape[0]-1,dataf.shape[0]-2]))
  dataf = pd.wide_to_long(dataf, ["H"], i="date", j="hour").reset_index().rename(columns={"H":"water_level"}).sort_values(by=['date', 'hour'])
  dataf.insert(2, 'datetime', pd.to_datetime(dataf['date'].astype(str) + ' ' +(dataf['hour']-1).astype(str) + ':00:00'))
  return (dataf
          .drop(columns=['hour', 'date'])
          )

def drop_bad_wl(dataf):
  dataf['water_level']=dataf['water_level'].astype(float)
  return (dataf
          .replace(0.0, np.nan)
          #.loc[(dataf['water_level']>0.0)]
          )


def add_station(dataf, name):
  dataf = dataf.rename(columns={'water_level':'water_level_'+name})
  return dataf

### Precipitations

In [ ]:
def prepare_df_preci(dataf):
  dataf = (dataf
          .rename(columns={"관측 일시":"date", "01시":"H01", "02시":"H02", "03시":"H03", "04시":"H04", "05시":"H05", "06시":"H06", "07시":"H07", "08시":"H08", "09시":"H09", "10시":"H10","11시":"H11", "12시":"H12", "13시":"H13", "14시":"H14", "15시":"H15", "16시":"H16", "17시":"H17", "18시":"H18", "19시":"H19", "20시":"H20", "21시":"H21", "22시":"H22", "23시":"H23", "24시":"H24"})
          .drop([dataf.shape[0]-1,dataf.shape[0]-2])
          )
  dataf = pd.wide_to_long(dataf, ["H"], i="date", j="hour").reset_index().rename(columns={"H":"precipitation"}).sort_values(by=['date', 'hour'])
  dataf.insert(2, 'datetime', pd.to_datetime(dataf['date'].astype(str) + ' ' +(dataf['hour']-1).astype(str) + ':00:00'))
  return (dataf
          .drop(columns=['hour', 'date'])
          )

def drop_bad_preci(dataf):
  dataf['precipitation']=dataf['precipitation'].astype(float)
  return (dataf
          .loc[(dataf['precipitation']>=0.0)])


def add_station_preci(dataf, name):
  dataf = dataf.rename(columns={'precipitation':'precipitation_'+name})
  return dataf


### Weather

In [ ]:
def prepare_df_temp(dataf):
  dataf = dataf.drop(columns=['Unnamed: 0'])
  dataf.insert(1, 'datetime', pd.to_datetime(df_temp['date'].astype(str)))
  dataf = dataf.drop(columns=['date'])
  return dataf

# General file preparation

### File paths

In [ ]:
base_drive_path = 'drive/MyDrive/flood/korea/'
xls_path_base = 'wl/'
wl_station = ['1007639','1007641','1007635','1007633','1007634','1007625']
xls_path_base_preci = base_drive_path + 'preci/'
preci_station = ['10074030_preci', '10074070_preci', '10054010_preci']

xls_path_base_temp = base_drive_path + 'temp/raw_temp_clean.xlsx'

## Data Imports

### Water levels

In [ ]:
first_ts = []
list_of_df = []
for station_id, station_name in enumerate(wl_station):
  xls_path = xls_path_base + station_name + '.xls'
  df = (pd.read_excel(xls_path)
  .pipe(prepare_df)
  .pipe(drop_bad_wl)
  .pipe(add_station, station_name)
  )
  df.reset_index(inplace=True, drop=True)
  df = df.set_index(df['datetime'])
  first_ts_val = df.values[0,0]
  list_of_df.append(df)
  first_ts.append(first_ts_val)

for df_id in range(len(list_of_df)):
  list_of_df[df_id] = list_of_df[df_id].loc[(list_of_df[df_id]['datetime']>max(first_ts))]
  if df_id:
    complete_df = pd.concat([complete_df, list_of_df[df_id]], axis=1)
  else:
    complete_df = list_of_df[df_id].copy()
complete_df_bu = complete_df.copy()
complete_df = complete_df.replace(r'^s*$', np.nan, regex = True)  # Replace blanks by NaN
complete_df=complete_df.drop(columns=['datetime'])
complete_df_inter = complete_df.copy().astype('float')
for col_name in complete_df_inter.columns:
  complete_df_inter[col_name] = complete_df_inter[col_name].interpolate()
complete_df_inter.dropna(inplace=True)
complete_df.dropna(inplace = True)


### Weather

In [ ]:
df_temp = pd.read_excel(xls_path_base_temp)
complete_df_temp = df_temp.pipe(prepare_df_temp)
complete_df_temp = complete_df_temp.loc[(complete_df_temp['datetime']>=complete_df_inter.index.min())]
complete_df_temp = complete_df_temp.loc[(complete_df_temp['datetime']<=complete_df_inter.index.max())]

complete_df_temp.reset_index(inplace=True, drop=True)
complete_df_temp = complete_df_temp.set_index(complete_df_temp['datetime'])
complete_df_temp = complete_df_temp.drop(columns=['datetime'])


### Precipitations

In [ ]:
first_ts_preci = []
list_of_df_preci = []
for station_id, station_name in enumerate(preci_station):
  xls_path_preci = xls_path_base_preci + station_name + '.xlsx'
  df_preci = (pd.read_excel(xls_path_preci)
    .pipe(prepare_df_preci)
    .pipe(drop_bad_preci)
    .pipe(add_station_preci, station_name)
  )
  df_preci.reset_index(inplace=True, drop=True)
  df_preci = df_preci.set_index(df_preci['datetime'])
  first_ts_val_preci = df_preci.values[0,0]
  list_of_df_preci.append(df_preci.copy())
  first_ts_preci.append(first_ts_val_preci)

for df_id in range(len(list_of_df_preci)):
  list_of_df_preci[df_id] = list_of_df_preci[df_id].loc[(list_of_df_preci[df_id]['datetime']>=(complete_df_inter.index.min()))]

  if df_id:
    complete_df_preci = pd.concat([complete_df_preci, list_of_df_preci[df_id]], axis=1)
  else:
    complete_df_preci = list_of_df_preci[df_id].copy()
complete_df_preci_bu = complete_df_preci.copy()
complete_df_preci = complete_df_preci.replace(r'^s*$', np.nan, regex = True)  # Replace blanks by NaN
complete_df_preci= complete_df_preci.drop(columns=['datetime'])

for col_name in complete_df_preci.columns:
  complete_df_preci[col_name] = complete_df_preci[col_name].interpolate()

complete_df_preci.dropna(inplace=True)
#complete_df.dropna(inplace = True)

### Checks

#### Sizes Checks

In [ ]:
print(complete_df_preci.index.difference(complete_df_inter.index))
print(complete_df_inter.index.difference(complete_df_preci.index))
print(complete_df_temp.index.difference(complete_df_inter.index))

print(complete_df_temp.shape)
print(complete_df_preci.shape)
print(complete_df_inter.shape)

#### Gaps checks

Water Level

In [ ]:
mask_inter = complete_df_inter.index.to_series().diff() > pd.Timedelta('01:00:00')
mask_inter[mask_int_preci].index

Precipitations

In [ ]:
mask_int_preci = complete_df_preci.index.to_series().diff() > pd.Timedelta('01:00:00')
mask_int_preci[mask_int_preci].index

Weather


In [ ]:
mask_int_temp = complete_df_temp.index.to_series().diff() > pd.Timedelta('01:00:00')
mask_int_temp[mask_int_preci].index

# Forecasting

## General Functions Definition

In [ ]:
def compute_losses(y_train, y_fit, y_test, y_pred):
  train_mae = mean_absolute_error(y_train, y_fit)
  test_mae = mean_absolute_error(y_test, y_pred)

  train_rmse = mean_squared_error(y_train, y_fit, squared=False)
  test_rmse = mean_squared_error(y_test, y_pred, squared=False)

  def nse(targets, predictions):
      return (1-(np.sum((targets-predictions)**2)/np.sum((targets-np.mean(targets, axis=0))**2))).mean(axis=0)

  train_nse = nse(y_train, y_fit)
  test_nse = nse(y_test, y_pred)

  print((f"Training:\n MAE: {train_mae:.2f} MSE: {train_rmse:.2f} NSE: {train_nse:.2f}"))
  print((f"Testing:\n MAE: {test_mae:.2f} MSE: {test_rmse:.2f} NSE: {test_nse:.2f}\n"))


In [ ]:
def prepare_TrainTest_forecast(reframed_train, reframed_test, predict_ts, past_ts, nbr_features):
  test = reframed_test.values
  train = reframed_train.values

  # split into input and outputs
  train_X, train_y = train[:, :-predict_ts*nbr_features], train[:, -predict_ts*nbr_features:]
  test_X, test_y = test[:, :-predict_ts*nbr_features], test[:, -predict_ts*nbr_features:]
  # reshape input to be 3D [samples, timesteps, features]
  train_X = train_X.reshape((train_X.shape[0], past_ts, nbr_features))
  test_X = test_X.reshape((test_X.shape[0], past_ts, nbr_features))

  return train_X, train_y, test_X, test_y

In [ ]:
def prepare_TrainTest_forecast_singleoutput(reframed_train, reframed_test, predict_ts, past_ts, nbr_features, pred_station):

  # split into input and outputs
  train_X = reframed_train.iloc[:, :(past_ts*nbr_features+1)].values
  train_y = reframed_train.iloc[:, (past_ts*nbr_features+1):].filter(regex=pred_station).values

  test_X = reframed_test.iloc[:, :(past_ts*nbr_features+1)].values
  test_y = reframed_test.iloc[:, (past_ts*nbr_features+1):].filter(regex=pred_station).values

  return train_X, train_y, test_X, test_y

In [ ]:
# convert series to supervised learning
def series_to_supervised(data_in, n_in=1, n_out=1, dropnan=True):
  data = data_in.copy()
  n_vars = data.shape[1]
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(data.shift(i))
    names += [('%s(t-%d)' % (data.columns[j], i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    cols.append(data.shift(-i))
    if i == 0:
      names += [('%s(t)' % (data.columns[j])) for j in range(n_vars)]
    else:
      names += [('%s(t+%d)' % (data.columns[j], i)) for j in range(n_vars)]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
    agg.dropna(inplace=True)
  return agg


In [ ]:
class MinMaxScale:

  def __init__(self, dataf, ul=1, ll=0):
    self.min = dataf.min()
    self.max = dataf.max()
    self.UL = ul
    self.LL = ll

  def NormMM(self, dataf):
    return self.LL+(self.UL-self.LL)*(dataf - self.min)/(self.max-self.min)

  def DeNormMM(self, dataf, station_filter):
    return (dataf - self.LL) * (self.max-self.min).filter(regex=station_filter).values[0]/(self.UL-self.LL) + self.min.filter(regex=station_filter).values[0]


## Linear models

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

## Classic regression

In [ ]:
train_ratio = 0.75
past_ts=72
forecast_ts_list=[2]#[i for i in range(1,13)]

predict_station_list = [0]
for predict_station_id in predict_station_list:
  for predict_ts in forecast_ts_list:
    predict_station = wl_station[predict_station_id]

    train_ts = complete_df.iloc[int(train_ratio*complete_df.shape[0])].name
    df_train = complete_df.loc[:train_ts].iloc[:,predict_station_id:]
    df_test = complete_df.loc[train_ts:].iloc[:,predict_station_id:]
    nbr_features = df_train.columns.unique().shape[0]

    # define minmax scaler
    scale_wl = MinMaxScale(df_train)

    # scale the train and test data
    df_train_scaled = scale_wl.NormMM(df_train)
    df_test_scaled = scale_wl.NormMM(df_test)

    reframed_train = series_to_supervised(df_train_scaled, past_ts, predict_ts)
    reframed_test = series_to_supervised(df_test_scaled, past_ts, predict_ts)

    train_X, train_y, test_X, test_y = prepare_TrainTest_forecast_singleoutput(reframed_train, reframed_test, predict_ts, past_ts, nbr_features,predict_station)

    model = LinearRegression()
    model.fit(train_X, train_y)

    y_train_scaled = reframed_train.iloc[:, (past_ts*nbr_features+1):].filter(regex=predict_station)
    y_test_scaled = reframed_test.iloc[:, (past_ts*nbr_features+1):].filter(regex=predict_station)
    y_fit_scaled = pd.DataFrame(model.predict(train_X), index=y_train_scaled.index, columns=y_train_scaled.columns)
    y_pred_scaled = pd.DataFrame(model.predict(test_X), index=y_test_scaled.index, columns=y_test_scaled.columns)

    y_train = scale_wl.DeNormMM(y_train_scaled, predict_station)
    y_test  = scale_wl.DeNormMM(y_test_scaled, predict_station)
    y_fit   = scale_wl.DeNormMM(y_fit_scaled, predict_station)
    y_pred  = scale_wl.DeNormMM(y_pred_scaled, predict_station)

    if predict_ts > 1:
      print(f'Prediction for station {predict_station} with a forecast of {predict_ts} hours and a lookback of {past_ts} hours.')
    else:
      print(f'Prediction for station {predict_station} with a forecast of {predict_ts} hour and a lookback of {past_ts} hours.')
    compute_losses(y_train, y_fit, y_test, y_pred)

In [ ]:
palette = dict(palette='husl', n_colors=64)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(11, 8))

plot_window_train = ["2015-01-01 01:00:00", "2015-01-02 0:00:00"]
plot_label_train = ["2015-01-01 00:00:00", "2015-01-02 10:00:00"]

plot_window_test = ["2022-01-01 01:00:00", "2022-01-02 0:00:00"]
plot_label_test = ["2022-01-01 00:00:00", "2022-01-02 10:00:00"]

y_fit_windowed = y_fit.loc[plot_window_train[0]:plot_window_train[1]]
y_pred_windowed = y_pred.loc[plot_window_test[0]:plot_window_test[1]]


# Plot training prediction
ax1= y_train.loc[plot_label_train[0]:plot_label_train[1]].iloc[:,0].plot(**plot_params, ax=ax1)

if forecast_ts_list[-1] > 2:
  ax1 = plot_multistep(y_fit_windowed, ax=ax1, palette_kwargs=palette)
else:
  ax1= y_fit_windowed.plot(**plot_params2, ax=ax1)

ax1.title.set_text(f'Training prediction: {past_ts}H lookback and {forecast_ts_list[-1]-1}H forecast')
ax2.title.set_text(f'Testing prediction: {past_ts}H lookback and {forecast_ts_list[-1]-1}H forecast')

# Plot testing prediction
if forecast_ts_list[-1] > 2:
  ax2 = plot_multistep(y_pred_windowed, ax=ax2, palette_kwargs=palette)
else:
  ax2= y_pred_windowed.plot(**plot_params2, ax=ax2)

ax2= y_test.loc[plot_label_test[0]:plot_label_test[1]].iloc[:,0].plot(**plot_params, ax=ax2)


## L2 Regularization

In [ ]:
train_ratio = 0.75
past_ts=72
forecast_ts_list=[2]#[i for i in range(1,13)]

predict_station_list = [0]
for predict_station_id in predict_station_list:
  for predict_ts in forecast_ts_list:
    predict_station = wl_station[predict_station_id]

    train_ts = complete_df.iloc[int(train_ratio*complete_df.shape[0])].name
    df_train = complete_df.loc[:train_ts].iloc[:,predict_station_id:]
    df_test = complete_df.loc[train_ts:].iloc[:,predict_station_id:]
    nbr_features = df_train.columns.unique().shape[0]

    # define minmax scaler
    scale_wl = MinMaxScale(df_train)

    # scale the train and test data
    df_train_scaled = scale_wl.NormMM(df_train)
    df_test_scaled = scale_wl.NormMM(df_test)

    reframed_train = series_to_supervised(df_train_scaled, past_ts, predict_ts)
    reframed_test = series_to_supervised(df_test_scaled, past_ts, predict_ts)

    train_X, train_y, test_X, test_y = prepare_TrainTest_forecast_singleoutput(reframed_train, reframed_test, predict_ts, past_ts, nbr_features,predict_station)

    ridge = Ridge()
    ridge.fit(train_X, train_y)

    y_train_scaled = reframed_train.iloc[:, (past_ts*nbr_features+1):].filter(regex=predict_station)
    y_test_scaled = reframed_test.iloc[:, (past_ts*nbr_features+1):].filter(regex=predict_station)
    y_fit_scaled = pd.DataFrame(ridge.predict(train_X), index=y_train_scaled.index, columns=y_train_scaled.columns)
    y_pred_scaled = pd.DataFrame(ridge.predict(test_X), index=y_test_scaled.index, columns=y_test_scaled.columns)

    y_train = scale_wl.DeNormMM(y_train_scaled, predict_station)
    y_test  = scale_wl.DeNormMM(y_test_scaled, predict_station)
    y_fit   = scale_wl.DeNormMM(y_fit_scaled, predict_station)
    y_pred  = scale_wl.DeNormMM(y_pred_scaled, predict_station)

    if predict_ts > 1:
      print(f'Prediction for station {predict_station} with a forecast of {predict_ts} hours and a lookback of {past_ts} hours.')
    else:
      print(f'Prediction for station {predict_station} with a forecast of {predict_ts} hour and a lookback of {past_ts} hours.')
    compute_losses(y_train, y_fit, y_test, y_pred)

In [ ]:
palette = dict(palette='husl', n_colors=64)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(11, 8))

plot_window_train = ["2015-01-01 01:00:00", "2015-01-02 0:00:00"]
plot_label_train = ["2015-01-01 00:00:00", "2015-01-02 10:00:00"]

plot_window_test = ["2022-01-01 01:00:00", "2022-01-02 0:00:00"]
plot_label_test = ["2022-01-01 00:00:00", "2022-01-02 10:00:00"]

y_fit_windowed = y_fit.loc[plot_window_train[0]:plot_window_train[1]]
y_pred_windowed = y_pred.loc[plot_window_test[0]:plot_window_test[1]]


# Plot training prediction
ax1= y_train.loc[plot_label_train[0]:plot_label_train[1]].iloc[:,0].plot(**plot_params, ax=ax1)

if forecast_ts_list[-1] > 2:
  ax1 = plot_multistep(y_fit_windowed, ax=ax1, palette_kwargs=palette)
else:
  ax1= y_fit_windowed.plot(**plot_params2, ax=ax1)

ax1.title.set_text(f'Training prediction: {past_ts}H lookback and {forecast_ts_list[-1]-1}H forecast')
ax2.title.set_text(f'Testing prediction: {past_ts}H lookback and {forecast_ts_list[-1]-1}H forecast')

# Plot testing prediction
if forecast_ts_list[-1] > 2:
  ax2 = plot_multistep(y_pred_windowed, ax=ax2, palette_kwargs=palette)
else:
  ax2= y_pred_windowed.plot(**plot_params2, ax=ax2)

ax2= y_test.loc[plot_label_test[0]:plot_label_test[1]].iloc[:,0].plot(**plot_params, ax=ax2)


## LSTM Models

## General functions

In [ ]:
class FLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=256, num_layers=2, forecast_length = 1):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, forecast_length)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

class LSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

In [ ]:


def define_model(train_X, train_y, input_size=1, hidden_size=256, num_layers=2, forecast_length = 1, lr=0.001, batch_size = 20)
  model = FLSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, forecast_length = forecast_length)
  loss_function = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=lr)

  # Create dataloader
  training_loader = data.DataLoader(data.TensorDataset(train_X, train_y), shuffle=True, batch_size=batch_size)

  return model, loss_function, optimizer, training_loader



def training_model(model, loss_function, optimizer, training_loader, train_X, train_y, test_X, test_y, nbr_epoch = 30, save_epoch = 10, save_path=None)
  for epoch in range(nbr_epoch):
      model.train()
      for X_batch, y_batch in training_loader:
          y_fit = model(X_batch)
          loss = loss_function(y_fit, y_batch)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      # Saves model
      if not (epoch + 1) % save_epoch and save_path is not None:
        savename_path = base_drive_path + 'weights/' + save_path + '_epoch_'+str(epoch+1)+'.pt'
        torch.save(model.state_dict(), savename_path) # official recommended

      # Validation
      if (epoch+1) % 5 != 0:
          continue
      model.eval()
      with torch.no_grad():
          y_fit = model(train_X)
          train_rmse = np.sqrt(loss_fn(y_fit, train_y))
          y_pred = model(test_X)
          test_rmse = np.sqrt(loss_fn(y_pred, test_y))
      print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))
  return y_fit, y_pred, model

In [ ]:
def prepare_TrainTest_forecast_singleoutput_LSTM(reframed_train, reframed_test, predict_ts, past_ts, nbr_features, pred_station):

  # split into input and outputs
  train_X = reframed_train.iloc[:, :(past_ts+1)*nbr_features].values
  train_y = reframed_train.iloc[:, (past_ts+1)*nbr_features:].filter(regex=pred_station).values

  test_X = reframed_test.iloc[:, :(past_ts+1)*nbr_features].values
  test_y = reframed_test.iloc[:, (past_ts+1)*nbr_features:].filter(regex=pred_station).values


  # reshape input to be 3D [samples, timesteps, features]
  train_X = train_X.reshape((train_X.shape[0], past_ts+1, nbr_features))
  test_X = test_X.reshape((test_X.shape[0], past_ts+1, nbr_features))

  return train_X, train_y, test_X, test_y

In [ ]:
def prepare_past_stages(reframed_train, reframed_test, predict_ts, past_ts, nbr_features, pred_station, predict_station_list):

  # split into input and outputs
  train_X = reframed_train.iloc[:, :(past_ts+1)*nbr_features]
  train_y = reframed_train.iloc[:, (past_ts+1)*nbr_features:]

  test_X = reframed_test.iloc[:, :(past_ts+1)*nbr_features]
  test_y = reframed_test.iloc[:, (past_ts+1)*nbr_features:]

  if predict_station_list is not None:
    past_train_x, past_test_x, past_train_y, past_test_y = None, None, None, None
    for id_filter in predict_station_list:

      tmp_train_x = (train_X.filter(regex=id_filter)).values
      tmp_train_y = (train_y.filter(regex=id_filter)).values

      tmp_test_x = (test_X.filter(regex=id_filter)).values
      tmp_test_y = (test_y.filter(regex=id_filter)).values

      if past_train_x is None:
        past_train_x = tmp_train_x.copy()
        past_train_y = tmp_train_y.copy()
        past_test_x = tmp_test_x.copy()
        past_test_y = tmp_test_y.copy()

      else:
        past_train_x = np.concatenate((past_train_x,tmp_train_x), axis=1)
        past_train_y = np.concatenate((past_train_y,tmp_train_y), axis=1)
        past_test_x = np.concatenate((past_test_x,tmp_test_x), axis=1)
        past_test_y = np.concatenate((past_test_y,tmp_test_y), axis=1)

    nbr_features = len(predict_station_list)+1


  else:
    past_train_x = train_X.drop(train_X.filter(regex=pred_station).columns,axis=1).values
    past_train_y = train_y.drop(train_y.filter(regex=pred_station).columns,axis=1).values

    past_test_x = test_X.drop(test_X.filter(regex=pred_station).columns,axis=1).values
    past_test_y = test_y.drop(test_y.filter(regex=pred_station).columns,axis=1).values

  train_X = train_X.filter(regex=pred_station).values
  train_y = train_y.filter(regex=pred_station).values
  test_X_pd = test_X.filter(regex=pred_station)
  test_X = test_X_pd.values
  test_y = test_y.filter(regex=pred_station).values

  # reshape input to be 3D [samples, timesteps, features]

  train_X = train_X.reshape((train_X.shape[0], past_ts+1, 1))
  past_train_x = past_train_x.reshape((past_train_x.shape[0], past_ts+1, nbr_features-1))

  test_X = test_X.reshape((test_X.shape[0], past_ts+1, 1))
  past_test_x = past_test_x.reshape((past_test_x.shape[0], past_ts+1, nbr_features-1))


  return train_X, train_y, test_X, test_y, past_train_x, past_train_y, past_test_x, past_test_y, test_X_pd

In [ ]:
import keras
# TODO complete this for correct prediction
train_ratio = 0.75
past_ts=19
forecast_ts_list=[2]#[i for i in range(1,13)]
train = True
norm_input = True
load_model = False

df_fused = pd.concat([complete_df_inter, complete_df_preci, complete_df_temp], axis=1)

fuse_input = True
other_station_id = None
predict_station_list = [0, 1, 2, 3]
for predict_station_id in predict_station_list:
  for predict_ts in forecast_ts_list:
    predict_station = wl_station[predict_station_id]

    if fuse_input:
      other_input_station = predict_station_list.copy()
      other_input_station.pop(predict_station_id)

      other_station_id = [wl_station[i] for i in other_input_station]

      other_station_id += preci_station
    train_ts = df_fused.iloc[int(train_ratio*df_fused.shape[0])].name
    df_train = df_fused.loc[:train_ts].iloc[:,predict_station_id:]
    df_test = df_fused.loc[train_ts:].iloc[:,predict_station_id:]
    nbr_features = df_train.columns.unique().shape[0]


    if norm_input:
      # define minmax scaler
      scale_wl = MinMaxScale(df_train, ul=1, ll=-1)

      # scale the train and test data
      df_train_scaled = scale_wl.NormMM(df_train)
      df_test_scaled = scale_wl.NormMM(df_test)
    else:
      df_train_scaled = df_train
      df_test_scaled = df_test

    reframed_train = series_to_supervised(df_train_scaled, past_ts, predict_ts)#.diff()[1:]
    reframed_test = series_to_supervised(df_test_scaled, past_ts, predict_ts)#.diff()[1:]
    train_X, train_y, test_X, test_y, past_train_x, past_train_y, past_test_x, past_test_y, test_X_pd = prepare_past_stages(reframed_train, reframed_test, predict_ts, past_ts, nbr_features, predict_station, other_station_id)

    if fuse_input:
      train_X = np.concatenate((train_X,past_train_x), axis=2)
      test_X = np.concatenate((test_X,past_test_x), axis=2)

    y_train_scaled = reframed_train.iloc[:, (past_ts*nbr_features+1):].filter(regex=predict_station)
    y_test_scaled = reframed_test.iloc[:, (past_ts*nbr_features+1):].filter(regex=predict_station)

    if train or load_model:
      model, loss_function, optimizer, training_loader = define_model(train_X, train_y, input_size=1, hidden_size=256, num_layers=2, forecast_length = predict_ts-1, lr=0.001, batch_size = 20)

      if load_model:
        model.load_state_dict(torch.load(base_drive_path + 'weights/training_model_epoch_20_300_hl.pt'))
        model.eval()
        with torch.no_grad():
            y_fit = model(train_X)
            y_pred = model(test_X)
      if train:
        savename = 'interpolated_preci_weather_one_forecast_two_input_training_model'
        y_fit, y_pred, model = training_model(model, loss_function, optimizer, training_loader, train_X, train_y, test_X, test_y, nbr_epoch = 30, save_epoch = 10, save_path=savename)

      y_fit_scaled = pd.DataFrame(y_fit, index=y_train_scaled.index, columns=y_train_scaled.columns)
      y_pred_scaled = pd.DataFrame(y_pred, index=y_test_scaled.index, columns=y_test_scaled.columns)
      if norm_input:
        y_fit   = scale_wl.DeNormMM(y_fit_scaled, predict_station)
        y_pred  = scale_wl.DeNormMM(y_pred_scaled, predict_station)
      else:
        y_fit = y_fit_scaled
        y_pred = y_pred_scaled

    if norm_input:
      y_train = scale_wl.DeNormMM(y_train_scaled, predict_station)
      y_test  = scale_wl.DeNormMM(y_test_scaled, predict_station)
    else:
      y_train = y_train_scaled
      y_test = y_test_scaled

    if train:
      if predict_ts > 1:
        print(f'Prediction for station {predict_station} with a forecast of {predict_ts-1} hours and a lookback of {past_ts} hours.')
      else:
        print(f'Prediction for station {predict_station} with a forecast of {predict_ts-1} hour and a lookback of {past_ts} hours.')
      compute_losses(y_train, y_fit, y_test, y_pred)

  break